In [25]:
# from model.LSTM_r import make_model
# from model.ASTGCN_full_r import make_model
# from model.LSTMGC_r import make_model
from model.LSTMGC_full_r import make_model
import torch
import torch.nn as nn
import torch.optim as optim
from lib.utils import get_adjacency_matrix
import numpy as np

In [13]:
DEVICE = torch.device('cpu')
print(DEVICE)

cpu


## Test LSTM

In [3]:
model = make_model(DEVICE, 3, 100, 1)

In [4]:
model

LSTM_submodule(
  (lstm): LSTM(3, 100)
  (linear): Linear(in_features=100, out_features=1, bias=True)
)

In [5]:
X_train = torch.rand((20, 30, 3, 12))
y_train = torch.rand((20, 30, 3))

In [6]:
criterion = nn.L1Loss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [7]:
outputs = model(X_train)

In [8]:
outputs.shape

torch.Size([20, 30, 12])

## Test ASTGCN

In [3]:
# adj_mx, distance_mx = get_adjacency_matrix('./data/PEMS04/PEMS04.csv', 
#                                            307, None)

In [3]:
adj_mx = np.array(torch.eye(30))

In [ ]:
# DEVICE, nb_block, in_channels, 
# K, nb_chev_filter, nb_time_filter, 
# time_strides, adj_mx, num_for_predict, 
# len_input, num_of_vertices):
model = make_model(DEVICE, nb_block=2, in_channels=1, K=3, 
                  nb_chev_filter=64, nb_time_filter=64, 
                  time_strides=1, adj_mx=adj_mx, num_for_predict=12, 
                  len_input=12, num_of_vertices=30)

In [ ]:
# (B, N_nodes, F_in, T_in)
X_h_train = torch.rand((20, 30, 1, 12))
X_d_train = torch.rand((20, 30, 1, 12))
X_w_train = torch.rand((20, 30, 1, 12))
y_train = torch.rand((20, 30, 1))

In [23]:
criterion = nn.L1Loss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [24]:
outputs = model(X_h_train, X_d_train, X_w_train)

In [25]:
outputs.shape

torch.Size([20, 30, 12])

In [40]:
a = torch.ones((583, 307, 3, 12))
b = torch.ones((583, 307, 3, 12)) + 1
c = torch.ones((583, 307, 3, 12)) + 2

In [41]:
a = torch.unsqueeze(a, dim=1)
b = torch.unsqueeze(b, dim=1)
c = torch.unsqueeze(c, dim=1)

In [42]:
a.shape, b.shape, c.shape

(torch.Size([583, 1, 307, 3, 12]),
 torch.Size([583, 1, 307, 3, 12]),
 torch.Size([583, 1, 307, 3, 12]))

In [47]:
d = torch.cat([a, b, c], dim=1)

In [ ]:
d[:1, :, :, :]

## LSTMGC

In [3]:
adj_mx = np.array(torch.ones(30, 30))

In [4]:
model = make_model(DEVICE, in_channels=1, out_feat=1, hidden_feat=50, adj_mx=adj_mx, num_for_predict=12, len_input=12, num_of_vertices=30)

In [5]:
X = torch.rand((20, 30, 1, 12))

In [6]:
criterion = nn.L1Loss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [8]:
X.shape

torch.Size([20, 12, 30, 1])

In [9]:
# (batch_size, input_seq_len, num_nodes, in_feat)
outputs = model(X)

torch.Size([30, 20, 12, 1])
gcn_out = torch.Size([30, 20, 12, 100])
gcn_out = torch.Size([600, 12, 100])


In [10]:
outputs.shape

torch.Size([20, 30, 12])

## LSTMGC Full

In [26]:
adj_mx = np.array(torch.ones(30, 30))

In [27]:
model = make_model(DEVICE, in_channels=1, out_feat=1, hidden_feat=50, adj_mx=adj_mx, num_for_predict=12, len_input=12, num_of_vertices=30)

In [28]:
model

LSTMGC_full_submodule(
  (h_model): LSTMGC_submodule(
    (graph_conv): GraphConv()
    (lstm): LSTM(100, 50, num_layers=2, batch_first=True)
    (dense): Linear(in_features=50, out_features=1, bias=True)
  )
  (d_model): LSTMGC_submodule(
    (graph_conv): GraphConv()
    (lstm): LSTM(100, 50, num_layers=2, batch_first=True)
    (dense): Linear(in_features=50, out_features=1, bias=True)
  )
  (w_model): LSTMGC_submodule(
    (graph_conv): GraphConv()
    (lstm): LSTM(100, 50, num_layers=2, batch_first=True)
    (dense): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [29]:
X = torch.rand((20, 3, 30, 1, 12))

In [30]:
criterion = nn.L1Loss().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
X.shape

torch.Size([20, 3, 30, 1, 12])

In [35]:
# (batch_size, input_seq_len, num_nodes, in_feat)
x1 = X[:, 0, :, :, :].squeeze(dim=1)
x2 = X[:, 1, :, :, :].squeeze(dim=1)
x3 = X[:, 2, :, :, :].squeeze(dim=1)
outputs = model(x1, x2, x3)

In [36]:
outputs.shape

torch.Size([20, 30, 12])